In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import os
import matplotlib.pyplot as plt
import json
from tqdm.notebook import tqdm
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
def get_adj_matrix(data_df):
    As = []
    for id in tqdm(data_df["id"]):
        a = np.load(f"{path}/bpps/{id}.npy")
        As.append(a)
    As = np.array(As)
    
    ## get adjacent matrix from structure sequence
    sequence_structure_adj = []
    for i in tqdm(range(len(data_df))):
        seq_length = data_df["seq_length"].iloc[i]
        structure = data_df["structure"].iloc[i]
        sequence = data_df["sequence"].iloc[i]

        cue = []
        a_structures = {
            ("A", "U") : np.zeros([seq_length, seq_length]),
            ("C", "G") : np.zeros([seq_length, seq_length]),
            ("U", "G") : np.zeros([seq_length, seq_length]),
            ("U", "A") : np.zeros([seq_length, seq_length]),
            ("G", "C") : np.zeros([seq_length, seq_length]),
            ("G", "U") : np.zeros([seq_length, seq_length]),
        }
        a_structure = np.zeros([seq_length, seq_length])
        for i in range(seq_length):
            if structure[i] == "(":
                cue.append(i)
            elif structure[i] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[i])][start, i] = 1
                a_structures[(sequence[i], sequence[start])][i, start] = 1
        
        a_strc = np.stack([a for a in a_structures.values()], axis = 2)
        a_strc = np.sum(a_strc, axis = 2, keepdims = True)
        sequence_structure_adj.append(a_strc)
    
    sequence_structure_adj = np.array(sequence_structure_adj)
    print(sequence_structure_adj.shape)
    
    ## adjacent matrix based on distance on the sequence
    ## D[i, j] = 1 / (abs(i - j) + 1) ** pow, pow = 1, 2, 4
    idx = np.arange(As.shape[1])
    distance_matrix = []
    for i in range(len(idx)):
        distance_matrix.append(np.abs(idx[i] - idx))

    distance_matrix = np.array(distance_matrix) + 1
    distance_matrix = 1/distance_matrix
    distance_matrix = distance_matrix[None, :,:]
    distance_matrix = np.repeat(distance_matrix, len(As), axis = 0)
    
    Dss = []
    for i in [1, 2, 4]: 
        Dss.append(distance_matrix ** i)
    distance_matrix = np.stack(Dss, axis = 3)
    print(distance_matrix.shape)
    
    adjacency_matrix = np.concatenate([As[:,:,:,None], sequence_structure_adj, distance_matrix], axis = 3).astype(np.float32)
    
    return adjacency_matrix
    
    

In [4]:
def get_node_features(train):
    ## get node features, which is one hot encoded
    mapping = {}
    vocab = ["A", "G", "C", "U"]
    for i, s in enumerate(vocab):
        mapping[s] = [0] * len(vocab)
        mapping[s][i] = 1
    X_node = np.stack(train["sequence"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))

    mapping = {}
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    for i, s in enumerate(vocab):
        mapping[s] = [0] * len(vocab)
        mapping[s][i] = 1
    X_loop = np.stack(train["predicted_loop_type"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    mapping = {}
    vocab = [".", "(", ")"]
    for i, s in enumerate(vocab):
        mapping[s] = [0] * len(vocab)
        mapping[s][i] = 1
    X_structure = np.stack(train["structure"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    
    X_node = np.concatenate([X_node, X_loop], axis = 2)
    
    ## interaction
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis = 2)
    vocab = sorted(set(a.flatten()))
    print(vocab)
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis = 2)
    X_node = np.concatenate([X_node, ohes], axis = 2).astype(np.float32)
    
    
    print(X_node.shape)
    return X_node



In [5]:
import tensorflow as tf
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

def mcrmse(t, p, seq_len_target):
    ## calculate mcrmse score by using numpy
    t = t[:, :seq_len_target]
    p = p[:, :seq_len_target]
    
    score = np.mean(np.sqrt(np.mean(np.mean((p - t) ** 2, axis = 1), axis = 0)))
    return score

def attention(x_inner, x_outer, n_factor, dropout):
    x_Q =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_inner)
    x_K =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_V =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att

def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor, 
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform',
                     )(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x

def res(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])

def forward(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h

def adj_attn(x, adj, unit, n = 2, rate = 0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.matmul(adj, x_a) ## aggregate neighborhoods
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(X_node, adj_matrix):
    ## base model architecture 
    ## node, adj -> middle feature
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, adj_matrix.shape[3]), name = "adj")
    
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis = 3)([adj, adj_learned])
        
    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel = 3, rate = 0.0)
    x2 = forward(x1, 64, kernel = 6, rate = 0.0)
    x3 = forward(x2, 32, kernel = 15, rate = 0.0)
    x4 = forward(x3, 16, kernel = 30, rate = 0.0)
    x = L.Concatenate()([x1, x2, x3, x4])
    
    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attn(x, adj_all[:, :, :, i], unit, rate = 0.0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel = 30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    return model


def get_ae_model(base, X_node, adj_matrix):
    ## denoising auto encoder part
    ## node, adj -> middle feature -> node
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, adj_matrix.shape[3]), name = "adj")

    x = base([L.SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64, rate = 0.3)
    p = L.Dense(X_node.shape[2], "sigmoid")(x)
    
    loss = - tf.reduce_mean(20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4))
    model = tf.keras.Model(inputs = [node, adj], outputs = [loss])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = lambda t, y : y)
    return model


# loss functions
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=(1))
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def get_model(base, X_node, adj_matrix, seq_len_target):
    ## regression part
    ## node, adj -> middle feature -> prediction of targets
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, adj_matrix.shape[3]), name = "adj")
    
    x = base([node, adj])
    x = forward(x, 128, rate = 0.4)
    x = L.Dense(5, None)(x)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    
    opt = get_optimizer()
    def mcrmse_loss(t, y, seq_len_target=seq_len_target):
        ## calculate mcrmse score by using tf
        t = t[:, :seq_len_target]
        y = y[:, :seq_len_target]

        loss = tf.reduce_mean(tf.sqrt(tf.reduce_mean(tf.reduce_mean((t - y) ** 2, axis = 1), axis = 0)))
        return loss
    model.compile(optimizer = opt, loss = mcrmse_loss)
    return model

def get_optimizer():
    adam = tf.optimizers.Adam()
    return adam

In [6]:
def train_auto_encoder(X_node, adjacency_matrix, epochs, epochs_each, batch_size, save_path):
    base = get_base(X_node, adjacency_matrix)
    ae_model = get_ae_model(base, X_node, adjacency_matrix)
    for i in range(epochs//epochs_each):
        print(f"------ {i} ------")
        ae_model.fit([X_node, adjacency_matrix], [X_node[:,0]],
                  epochs = epochs_each,
                  batch_size = batch_size)
#         ae_model.fit([X_node_pub, As_pub], [X_node_pub[:,0]],
#                   epochs = epochs_each,
#                   batch_size = batch_size)
#         ae_model.fit([X_node_pri, As_pri], [X_node_pri[:,0]],
#                   epochs = epochs_each,
#                   batch_size = batch_size)
        gc.collect()
    print("****** save ae model ******")
    base.save_weights(save_path)
    

In [7]:
from sklearn.model_selection import KFold
def train_gcn(X_node, adjacency_matrix, seq_len_target, epochs, batch_size,
              model_path, ae_model_path=None, n_fold=2, validation_frequency=1):
    kfold = KFold(n_fold, shuffle = True, random_state = 42)

    legends = []
    for fold, (tr_idx, va_idx) in enumerate(kfold.split(X_node, adjacency_matrix)):

        gc.collect()
        tf.keras.backend.clear_session()
        print(f'\nFold - {fold}\n')
        X_node_tr = X_node[tr_idx]
        X_node_va = X_node[va_idx]
        As_tr = adjacency_matrix[tr_idx]
        As_va = adjacency_matrix[va_idx]
        y_tr = y[tr_idx]
        y_va = y[va_idx]  

        base = get_base(X_node, adjacency_matrix)
        if ae_model_path:
            base.load_weights(ae_model_path)
        model = get_model(base, X_node, adjacency_matrix, seq_len_target)
        filepath_list = model_path.split('.')
        filepath = filepath_list[0] + f'_{fold}' + filepath_list[1]
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                    filepath=filepath, save_weights_only=True,
                                    monitor='val_loss',mode='min',save_best_only=True)

        history = model.fit([X_node_tr, As_tr], [y_tr],
                            validation_data=([X_node_va, As_va], [y_va]),
                            epochs = epochs,
                            batch_size = batch_size, 
                            validation_freq = validation_frequency,
                            callbacks=[model_checkpoint_callback]
                       )

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        legends.append(f'loss_fold_{fold}')
        legends.append(f'val_loss_fold_{fold}')

    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(legends, loc='upper left')
    plt.show()

In [8]:
denoise = True # if True, use train data whose signal_to_noise > 1

path = "../input/stanfordcovidvaccine"

train = pd.read_json(f"{path}/train.json",lines=True)
if denoise:
    train = train[train.signal_to_noise > 1].reset_index(drop = True)
    
test  = pd.read_json(f"{path}/test.json",lines=True)
test_pub = test[test["seq_length"] == 107]
test_pri = test[test["seq_length"] == 130]
sub = pd.read_csv(f"{path}/sample_submission.csv")

targets = list(sub.columns[1:])
print(targets)

y_train = []
seq_len = train["seq_length"].iloc[0]
seq_len_target = train["seq_scored"].iloc[0]
ignore = -10000
ignore_length = seq_len - seq_len_target
for target in targets:
    y = np.vstack(train[target])
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore
    y = np.hstack([y, dummy])
    y_train.append(y)
y = np.stack(y_train, axis = 2)
y.shape

adjacency_matrix = get_adj_matrix(train)
adjacency_matrix_pub = get_adj_matrix(test_pub)
adjacency_matrix_pri = get_adj_matrix(test_pri)

X_node = get_node_features(train)
X_node_pub = get_node_features(test_pub)
X_node_pri = get_node_features(test_pri)



In [ ]:

epochs_list = [30, 10, 3, 3, 5, 5]
batch_size_list = [8, 16, 32, 64, 128, 256] 

epochs = epochs_list[0]
batch_size = batch_size_list[1]

model_path = "./model_without_ae.h5"
train_gcn(X_node, adjacency_matrix, seq_len_target, epochs, batch_size,
              model_path)

In [14]:
ae_epochs = 20 # epoch of training of denoising auto encoder
ae_epochs_each = 5 # epoch of training of denoising auto encoder each time.                  
ae_batch_size = 32
ae_path = "./base_ae"
train_auto_encoder(X_node, adjacency_matrix, ae_epochs, ae_epochs_each, 
                   ae_batch_size, ae_path)

In [17]:
epochs_list = [30, 10, 3, 3, 5, 5]
batch_size_list = [8, 16, 32, 64, 128, 256] 

epochs = epochs_list[0]
batch_size = batch_size_list[1]

model_path = "./model_with_ae.h5"
train_gcn(X_node, adjacency_matrix, seq_len_target, epochs, batch_size,
              model_path, ae_model_path=ae_path)

In [ ]:


p_pub = 0
p_pri = 0
base = get_base(X_node, adjacency_matrix_pub)
model = get_model(base, X_node, adjacency_matrix_pub, seq_len_target)
for i in range(2):
    model.load_weights(f'./model_with_ae_{i}.h5')
    p_pub += model.predict([X_node_pub, adjacency_matrix_pub]) / 5
    p_pri += model.predict([X_node_pri, adjacency_matrix_pri]) / 5
    if one_fold:
        p_pub *= 5
        p_pri *= 5
        break

for i, target in enumerate(targets):
    test_pub[target] = [list(p_pub[k, :, i]) for k in range(p_pub.shape[0])]
    test_pri[target] = [list(p_pri[k, :, i]) for k in range(p_pri.shape[0])]

